In [15]:
import sys
import open3d as o3d
import numpy as np
sys.path.append('../')

from utils.data_utils import *
from utils.visualize import visualize_pcd




In [16]:
def compute_local_intensity_gradient(pcd_np: np.ndarray, radius: float = 0.1, min_neighbors: int = 3) -> np.ndarray:
    """
    各点の近傍領域における輝度勾配（Intensity Gradient）を計算し、
    変化の大きさと方向を特徴量として返す関数です。
    
    入力:
      pcd_np: (N, 4) の numpy 配列（各行は [x, y, z, intensity]）
      radius: 近傍探索の半径
      min_neighbors: 勾配計算に必要な最小近傍数（通常は3以上）
      
    出力:
      特徴量の numpy 配列（形状は (4, N)）
      各列は [勾配の大きさ, 勾配方向の x, y, z 成分] を示す。
    """
    intensities = pcd_np[:, 3].astype(np.float32)
    points = pcd_np[:, :3].astype(np.float32)
    N = points.shape[0]
    features = np.zeros((N, 4), dtype=np.float32)
    
    # Open3D の点群生成と KDTree の構築
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    kdtree = o3d.geometry.KDTreeFlann(pcd)
    
    for i in range(N):
        query_point = points[i]
        [k, idx, _] = kdtree.search_radius_vector_3d(query_point, radius)
        # 自分自身は除外する
        idx = [j for j in idx if j != i]
        if len(idx) < min_neighbors:
            continue  # 近傍点が少なければ勾配はゼロとする
        # 近傍点との相対座標（A）と輝度差（b）を計算
        A = points[idx] - query_point  # shape: (n, 3)
        b = intensities[idx] - intensities[i]  # shape: (n,)
        # 最小二乗法により勾配ベクトル g = [g_x, g_y, g_z] を推定
        g, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
        mag = np.linalg.norm(g)
        if mag > 1e-8:
            direction = g / mag
        else:
            direction = np.zeros(3, dtype=np.float32)
        features[i] = np.hstack([mag, direction])
    # Open3D の特徴量は (feature_dimension, number_of_points) の形状であるため転置して返す
    return features.T

def apply_ransac_with_gradient(src_pcd_np: np.ndarray, tgt_pcd_np: np.ndarray,
                               distance_threshold: float = 0.05,
                               ransac_n: int = 4,
                               num_iterations: int = 1000,
                               gradient_radius: float = 0.1,
                               min_neighbors: int = 3):
    """
    局所輝度勾配を特徴量として用い、RANSAC による位置合わせを実施する関数。
    
    入力:
      src_pcd_np, tgt_pcd_np: (N,4) の numpy 配列（各行は [x, y, z, intensity]）
      distance_threshold: 対応点間の最大許容距離
      ransac_n: 対応点サンプル数
      num_iterations: RANSAC の反復回数
      gradient_radius: 近傍探索の半径（輝度勾配計算用）
      min_neighbors: 勾配計算に必要な最小近傍数
      
    出力:
      推定された変換行列、fitness、inlier_rmse を返す。
    """
    # 座標抽出
    src_xyz = src_pcd_np[:, :3].astype(np.float32)
    tgt_xyz = tgt_pcd_np[:, :3].astype(np.float32)
    
    src_pcd = o3d.geometry.PointCloud()
    src_pcd.points = o3d.utility.Vector3dVector(src_xyz)
    tgt_pcd = o3d.geometry.PointCloud()
    tgt_pcd.points = o3d.utility.Vector3dVector(tgt_xyz)
    
    # 局所輝度勾配特徴量を計算
    src_feature = o3d.pipelines.registration.Feature()
    src_feature.data = compute_local_intensity_gradient(src_pcd_np, radius=gradient_radius, min_neighbors=min_neighbors)
    tgt_feature = o3d.pipelines.registration.Feature()
    tgt_feature.data = compute_local_intensity_gradient(tgt_pcd_np, radius=gradient_radius, min_neighbors=min_neighbors)
    
    # mutual_filter を True に設定して RANSAC による特徴マッチング
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        src_pcd, tgt_pcd,
        src_feature, tgt_feature,
        True,  # mutual_filter の指定
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        ransac_n,
        [o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
         o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)],
        o3d.pipelines.registration.RANSACConvergenceCriteria(num_iterations, 1000)
    )
    return result.transformation, result.fitness, result.inlier_rmse

def test_ransac_gradient_parameters(src_pcd_np: np.ndarray, tgt_pcd_np: np.ndarray, T_true: np.ndarray,
                                    distance_thresholds=[0.03, 0.05, 0.07],
                                    iterations_list=[50000],
                                    gradient_radii=[0.05, 0.1, 0.2],
                                    min_neighbors_list=[3, 5]):
    """
    局所輝度勾配特徴量を用いた RANSAC のパラメータ（distance_threshold, iterations,
    gradient_radius, min_neighbors）の組み合わせを実験し、各組み合わせの登録誤差を出力する関数です。
    """
    param_results = []
    for dt in distance_thresholds:
        for num_iter in iterations_list:
            for gr in gradient_radii:
                for mn in min_neighbors_list:
                    T_ransac, fitness, inlier_rmse = apply_ransac_with_gradient(
                        src_pcd_np, tgt_pcd_np,
                        distance_threshold=dt,
                        num_iterations=num_iter,
                        gradient_radius=gr,
                        min_neighbors=mn
                    )
                    loss = compute_simple_loss(T_ransac, T_true)
                    param_results.append((dt, num_iter, gr, mn, loss, T_ransac, fitness, inlier_rmse))
                    print(f"Gradient RANSAC - dt: {dt}, iter: {num_iter}, grad_radius: {gr}, min_neighbors: {mn}, "
                          f"Loss: {loss:.6f}, Fitness: {fitness:.6f}, RMSE: {inlier_rmse:.6f}")
    best = min(param_results, key=lambda x: x[4])
    print("\nBest parameters for gradient features:")
    print(f"  dt: {best[0]}, iter: {best[1]}, grad_radius: {best[2]}, min_neighbors: {best[3]}, loss: {best[4]:.6f}")
    return best  # (dt, num_iter, gr, min_neighbors, loss, T_ransac, fitness, rmse)

In [17]:
if __name__ == '__main__':
    # ファイルパス（必要に応じて変更してください）
    src_icn_file = '../ply/src_icn.ply'
    tgt_icn_file = '../ply/tgt_icn.ply'
    
    # load_ply は intensity=True により (N,4) の numpy 配列（各行は [x, y, z, intensity]）を返すと仮定
    src_icn = load_ply(src_icn_file, intensity=True)
    tgt_icn = load_ply(tgt_icn_file, intensity=True)
    
    # ダウンサンプリングと正規化（RANSAC用のデータ）
    downsample_point = 2048
    src_icn_ds = fps_downsample(src_icn, downsample_point, intensity=True)
    tgt_icn_ds = fps_downsample(tgt_icn, downsample_point, intensity=True)
    src_icn_ds = normalize_pc(src_icn_ds, intensity=True)
    tgt_icn_ds = normalize_pc(tgt_icn_ds, intensity=True)
    print("Downsampled shape:", src_icn_ds.shape)
    
    # 正解の 4x4 剛体変換行列（例）
    T_true = np.array([[9.99638133e-01, 1.55978225e-02, 2.19159857e-02, -4.11244323e-02],
                       [-1.90081220e-02, 9.86076289e-01, 1.65203643e-01, 2.98080257e-02],
                       [-1.90340168e-02, -1.65560443e-01, 9.86015946e-01, 1.01116551e-04],
                       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
    
    print("Testing RANSAC registration using local intensity gradient features...")
    best_params = test_ransac_gradient_parameters(src_icn_ds, tgt_icn_ds, T_true)
    best_T_ransac = best_params[5]
    
    rot_err, trans_err = compute_transformation_errors(best_T_ransac, T_true)
    print(f"\nRANSAC Rotation error: {rot_err:.6f}")
    print(f"RANSAC Translation error: {trans_err:.6f}")
    print('Best T_ransac:', best_T_ransac)
    
    loss = compute_simple_loss(best_T_ransac, T_true)
    print(f"Loss: {loss:.6f}")
    
    # ----------------------------------------------------------------------------
    # 可視化: 最適な RANSAC 変換を元の（正規化済み）全点群に適用して位置合わせ結果を表示
    # ----------------------------------------------------------------------------
    src_full_file = '../ply/lab1.ply'
    tgt_full_file = '../ply/lab2.ply'
    
    original_src = load_ply(src_full_file, intensity=False)
    original_tgt = load_ply(tgt_full_file, intensity=False)
    original_src = normalize_pc(original_src, intensity=False)
    original_tgt = normalize_pc(original_tgt, intensity=False)
    
    src_full_h = np.hstack((original_src, np.ones((original_src.shape[0], 1))))
    src_trans_full = (best_T_ransac @ src_full_h.T).T[:, :3]
    
    print("Applying best RANSAC transformation on full point clouds for visualization...")
    visualize_pcd(src_trans_full, original_tgt, mode='test', normalize=True)


TypeError: load_ply() got an unexpected keyword argument 'intensity'